## web scraping

In [ ]:

como cria teste unitario desse codigo abaixo?
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

def trata_html(imput):
    try:
        imput = imput.decode("utf-8")
    except:
        return ""
    return " ".join(imput.split()).replace("> <", "><")


def get_html_soup(url):
    reposnse = urlopen(url)
    html = reposnse.read()
    html = trata_html(html)
    soup = BeautifulSoup(html, "html.parser")
    return soup


def get_opcoes(soup, parameters):
    opcoes = {}
    btn_opt = soup.findAll("button", parameters)
    for btn in btn_opt:
        opcoes[btn.get("value")] = btn.getText()
    return opcoes


def get_periodo(soup):
    periodo = soup.find("label", {"class": "lbl_pesq"}).getText()
    periodo = [int(x) for x in re.findall('\d+', periodo)]
    return periodo


def get_table(soup, param):
    table = soup.findAll("table", {"class":"tb_base tb_dados"})[0]
    rows = table.findAll("tr")
    header = rows[0].findAll("th")
    csv_rows=[]
    for row in rows:
        csv_row = {}
        for idx, values in enumerate(row.findAll(["td","th"])):
            csv_row[header[idx].get_text().split("(")[0].rstrip().lstrip().lower().replace('sem definição', 'sem_definicao')] = values.get_text().rstrip().lstrip()
        csv_rows.append({**csv_row, **param})        
    return csv_rows


def flatten(xss):
    return [x for xs in xss for x in xs]


def save_csv(table, filename):
    df = pd.DataFrame(table).fillna('-')
    df.to_csv(filename, index=False,sep=";")


URL = "http://vitibrasil.cnpuv.embrapa.br/index.php"
soup = get_html_soup(URL)
opcoes = get_opcoes(soup, {"class": "btn_opt"})
opcoes.pop('opt_01')
opcoes.pop('opt_07')

sub_opcoes = get_opcoes(soup, {"class": "btn_sopt"})


#for key, value in opcoes.items():
for key, value in {'opt_03': 'Processamento'}.items():
    tables=[]
    url_opcao = f"?opcao={key}"
    _URL = URL + url_opcao
    print(_URL)
    soup = get_html_soup(_URL)
    periodo = get_periodo(soup)
    sub_opcoes = get_opcoes(soup, {"class": "btn_sopt"})
    print(value, periodo, sub_opcoes)
    
    for ano in list(range(periodo[0], periodo[1]+1)):

        param ={'ano':ano}
        url_ano = f"&ano={ano}"
        
        if len(sub_opcoes)>0:
            for k, v in sub_opcoes.items():
                param ={'ano':ano, 'opcao':v}
                url_subopcao = f"&subopcao={k}"
                _URL = URL + url_opcao + url_ano + url_subopcao
                soup = get_html_soup(_URL)
                table = get_table(soup, param)
                tables.append(table)
                print(_URL)
        else:
            _URL = URL + url_opcao + url_ano
            soup = get_html_soup(_URL)
            table = get_table(soup, param)
            tables.append(table)
            print(_URL)

    table_flatten = flatten(tables)
    save_csv(table_flatten,f'data/{value}.csv')

## api

In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel
from typing import List
import pandas as pd
import os
import jwt
from passlib.context import CryptContext
from datetime import datetime, timedelta

# Caminhos dos arquivos CSV
DATA_DIR = "../data/"
PRODUCAO_FILE = os.path.join(DATA_DIR, "Produção.csv")
PROCESSAMENTO_FILE = os.path.join(DATA_DIR, "Processamento.csv")
COMERCIALIZACAO_FILE = os.path.join(DATA_DIR, "Comercialização.csv")
IMPORTACAO_FILE = os.path.join(DATA_DIR, "Importação.csv")
EXPORTACAO_FILE = os.path.join(DATA_DIR, "Exportação.csv")

# Secret Key para assinar os tokens JWT
SECRET_KEY = "secret_key"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30

app = FastAPI()


# Classes de modelos de dados
class Producao(BaseModel):
    produto: str
    quantidade: str
    ano: int


class Processamento(BaseModel):
    cultivar: str
    quantidade: str
    ano: int
    opcao: str
    sem_definicao: str


class Comercializacao(BaseModel):
    produto: str
    quantidade: str
    ano: int


class Importacao(BaseModel):
    países: str
    quantidade: str
    valor: str
    ano: int
    opcao: str


class Exportacao(BaseModel):
    países: str
    quantidade: str
    valor: str
    ano: int
    opcao: str


# Classe para leitura de arquivos CSV
class CSVReader:
    @staticmethod
    def ler_csv(filename):
        df = pd.read_csv(filename, sep=";")
        return df.to_dict(orient="records")


# Segurança e Autenticação
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="/token")

# Simulação de um usuário
fake_users_db = {
    "user": {
        "username": "user",
        "full_name": "User",
        "email": "user@example.com",
        "hashed_password": "$2b$12$RX6xbfaxBBzWlEGkHjv1xeSbtFfhidNd.zPqp4VGCA4FPFcl1Fvji",  # senha: password
        "disabled": False,
    }
}


# Funções de autenticação
def verify_password(plain_password, hashed_password):
    return pwd_context.verify(plain_password, hashed_password)


def get_user(db, username: str):
    if username in db:
        user_dict = db[username]
        return user_dict


def authenticate_user(fake_db, username: str, password: str):
    user = get_user(fake_db, username)
    if not user:
        return False
    if not verify_password(password, user["hashed_password"]):
        return False
    return user


def create_access_token(data: dict, expires_delta: timedelta):
    to_encode = data.copy()
    expire = datetime.utcnow() + expires_delta
    to_encode.update({"exp": expire})
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt


# Rota de login para obter o token JWT
@app.post("/token")
async def login_for_access_token(form_data: OAuth2PasswordRequestForm = Depends()):
    user = authenticate_user(fake_users_db, form_data.username, form_data.password)
    if not user:
        raise HTTPException(
            status_code=401,
            detail="Invalid username or password",
            headers={"WWW-Authenticate": "Bearer"},
        )
    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(
        data={"sub": user["username"]}, expires_delta=access_token_expires
    )
    return {"access_token": access_token, "token_type": "bearer"}


# Função para decodificar e verificar o token JWT
async def get_current_user(token: str = Depends(oauth2_scheme)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        if username is None:
            raise HTTPException(
                status_code=401,
                detail="Could not validate credentials",
                headers={"WWW-Authenticate": "Bearer"},
            )
    except jwt.ExpiredSignatureError:
        raise HTTPException(
            status_code=401,
            detail="Token has expired",
            headers={"WWW-Authenticate": "Bearer"},
        )
    except jwt.JWTError:
        raise HTTPException(
            status_code=401,
            detail="Could not validate credentials",
            headers={"WWW-Authenticate": "Bearer"},
        )
    user = get_user(fake_users_db, username)
    if user is None:
        raise HTTPException(
            status_code=401,
            detail="User not found",
            headers={"WWW-Authenticate": "Bearer"},
        )
    return user


# Rotas protegidas com autenticação JWT
@app.get("/producao/", response_model=List[Producao])
async def listar_producao(current_user: dict = Depends(get_current_user)):
    return CSVReader.ler_csv(PRODUCAO_FILE)


@app.get("/processamento/", response_model=List[Processamento])
async def listar_processamento(current_user: dict = Depends(get_current_user)):
    return CSVReader.ler_csv(PROCESSAMENTO_FILE)


@app.get("/comercializacao/", response_model=List[Comercializacao])
async def listar_comercializacao(current_user: dict = Depends(get_current_user)):
    return CSVReader.ler_csv(COMERCIALIZACAO_FILE)


@app.get("/importacao/", response_model=List[Importacao])
async def listar_importacao(current_user: dict = Depends(get_current_user)):
    return CSVReader.ler_csv(IMPORTACAO_FILE)


@app.get("/exportacao/", response_model=List[Exportacao])
async def listar_exportacao(current_user: dict = Depends(get_current_user)):
    return CSVReader.ler_csv(EXPORTACAO_FILE)


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)
